# Assessment 2

In [16]:
# importing modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [44]:
#reading in the data for mhd
mhd_ch4 = pd.read_csv('data/mhd_ch4.csv')
mhd_co2= pd.read_csv('data/mhd_co2.csv')
mhd_n2o = pd.read_csv('data/mhd_n2o.csv')

#reading in data for tac
tac_ch4 = pd.read_csv('data/tac_ch4.csv')
tac_co2= pd.read_csv('data/tac_co2.csv')
tac_n2o = pd.read_csv('data/tac_n2o.csv')

#reading in species information
spec_info = pd.read_csv('data/species_info.csv')  
print(spec_info)
#creating dictionaries to make it easier to access data
tac_mhd = {'tac_ch4': tac_ch4, 'tac_co2': tac_co2, 'tac_n2o': tac_n2o, 'mhd_ch4':mhd_ch4, 'mhd_co2':mhd_co2, 'mhd_n2o':mhd_n2o}



  species scale units
0     co2  noaa   ppm
1     ch4   sio   ppb
2     n2o   sio   ppb


In [18]:
# class Data:
#  def __init__(self,data_tac,data_mhd):
#     self.data_tac = data_tac
#     self.data_mhd = data_mhd
#  def tac_metadata(self):
#     print(self.data_mhd)
# data = Data(data_tac['tac_ch4'], data_mhd['mhd_ch4'])
# data.tac_metadata()

In [48]:
class Data:
 def __init__(self,data: dict, file: str, info):
    self.info = info
    self.tac_ch4 = data['tac_ch4']
    self.tac_co2 = data['tac_co2']
    self.tac_n2o = data['tac_n2o']
    self.file = file
    
 def tac_metadata(self):
   if self.file[:3] == 'tac':
      print('The location of this data set is Tacolneston, East Anglia.\nThe site code is TAC')
   if self 
      print(self.file[4:])
   
   if self.file[:3] == 'mhd':
      print('The location of this data set is Mace Head, W. coast of the Republic of Ireland.\nThe site code is MHD')
   
   if self.file == 'tac_ch4':
      print(self.tac_ch4)
   if self.file == 'tac_co2':
      print(self.tac_co2)
   if self.file == 'tac_n2o':
      print(self.tac_n2o)
   
data_tac = Data(tac_mhd, 'tac_co2',spec_info)
data_tac.tac_metadata()

The location of this data set is Tacolneston, East Anglia.
The site code is TAC
co2
                     time         mf
0     2019-01-01 00:00:00  415.11685
1     2019-01-01 01:00:00  415.13544
2     2019-01-01 02:00:00  415.29680
3     2019-01-01 03:00:00  415.18658
4     2019-01-01 04:00:00  415.20870
...                   ...        ...
8755  2019-12-31 19:00:00  422.56314
8756  2019-12-31 20:00:00  425.81052
8757  2019-12-31 21:00:00  418.67838
8758  2019-12-31 22:00:00  422.47363
8759  2019-12-31 23:00:00  425.63577

[8760 rows x 2 columns]


In [27]:
# class Data:
#  def __init__(self,data: dict):
#     self.tac_ch4 = data['tac_ch4']
    
#  def tac_metadata(self,species):
#     print(self.species)
# data_tac = Data(tac_mhd)
# data.tac_metadata(tac_mhd['tac_ch4'])

TypeError: Data.tac_metadata() takes 1 positional argument but 2 were given